In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from openpyxl import load_workbook
import os

In [6]:
# URL del listado de productos
url = 'https://vehiculos.mercadolibre.com.mx/carros-usados_Desde_193_ITEM*CONDITION_2230581_NoIndex_True'

# Encabezados para la solicitud
encabezados = {
    "User -Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Realizar la solicitud
response = requests.get(url, headers=encabezados)
content = response.content
soup = BeautifulSoup(content, 'html.parser')

# Verificar si la conexión es exitosa
if response.status_code == 200:
    print("Conexión exitosa con la página")
else:
    print("Error al conectar con la página")

# Listas para almacenar datos
nombres = []
precios = []

# Extraer productos
productos = soup.find_all('h2', class_='poly-box poly-component__title')

for producto in productos:
    # Extraer el nombre
    nombre = producto.find('a')
    if nombre:
        nombres.append(nombre.text.strip())
    
    # Extraer el precio
    precio_div = producto.find_next('div', class_='poly-component__price')
    if precio_div:
        precio_span = precio_div.find('span', class_='andes-money-amount andes-money-amount--cents-superscript')
        if precio_span:
            precio = precio_span.find('span', class_='andes-money-amount__fraction')
            if precio:
                precios.append(precio.text.strip())

# Verificar las longitudes de las listas
print(f"Productos encontrados: {len(nombres)}")
print(f"Precios encontrados: {len(precios)}")

# Asegurarse de que las listas tengan la misma longitud
min_length = min(len(nombres), len(precios))
nombres = nombres[:min_length]
precios = precios[:min_length]

# Crear DataFrame
df_total = pd.DataFrame({
    'Nombre Producto': nombres,
    'Precio': precios
})


file_path = "productos_mercadolibre.xlsx"

# Comprobar si el archivo ya existe
if os.path.exists(file_path):
    # Leer el archivo existente
    data = pd.read_excel(file_path)
    
    # Asegurarse de que los nombres de las columnas estén en formato string y quitar espacios
    data.columns = data.columns.astype(str).str.strip()
    
    # Verificar si 'Precio final' está en el DataFrame antes de ordenar
    if 'Precio final' in data.columns:
        # Ordenar por columna "Precio final"
        data = data.sort_values(by='Precio final', ascending=True)
    else:
        print("Advertencia: La columna 'Precio final' no se encontró en el archivo existente.")
else:
    print("El archivo no existe, creando uno nuevo.")
    data = pd.DataFrame(columns=['Nombre Producto', 'Precio inicial', 'Precio final'])

# Suponiendo que tienes un DataFrame nuevo llamado df_total con los datos actuales
# Agregar los datos nuevos al archivo existente
data = pd.concat([data, df_total], ignore_index=True)

# Escribir el DataFrame actualizado y ordenado de nuevo en el archivo, manteniendo los decimales completos
with pd.ExcelWriter(file_path, engine='openpyxl', mode='w') as writer:
    data.to_excel(writer, index=False, float_format="%.2f")

print("Datos agregados y archivo actualizado correctamente.")

Conexión exitosa con la página
Productos encontrados: 48
Precios encontrados: 48
Advertencia: La columna 'Precio final' no se encontró en el archivo existente.
Datos agregados y archivo actualizado correctamente.
